In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn import * 

In [16]:
df = pd.read_csv("/data/kaggle/house-prices/data_combined_cleaned.csv")
df = df[~df.SalesPrice.isna()]
del df["Id"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [9]:
target = "SalesPrice"
X = df.copy()
del X[target]

X_dummy = pd.get_dummies(X)
y = np.log(df.SalesPrice)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy,
                            y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("scaler", preprocessing.StandardScaler()),
    ("poly", preprocessing.PolynomialFeatures(degree = 2, include_bias=False)),
    ("est", linear_model.LinearRegression())
])

pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
r2_score = metrics.r2_score(y_train, y_train_pred)
print("Train rmse", rmse, "training r2_score", r2_score)

y_test_pred = pipe.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_score = metrics.r2_score(y_test, y_test_pred)
print("Test rmse", rmse, "test r2_score", r2_score)

Train rmse 4.052469544908295e-15 training r2_score 1.0
Test rmse 0.21473830430950336 test r2_score 0.7560047449535685


Observation: training and test performance are significantly different. I want to regularize the models to find optimum performance - training and test performance should be close.  

In [19]:
target = "SalesPrice"
X = df.copy()
del X[target]

X_dummy = pd.get_dummies(X)
y = np.log(df.SalesPrice)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy,
                            y, test_size = 0.3, random_state = 1)


pipe = pipeline.Pipeline([
    ("scaler", preprocessing.StandardScaler()),
    ("poly", preprocessing.PolynomialFeatures(degree = 1, 
                                    include_bias=False)),
    ("est", linear_model.Lasso(alpha=0.1))
])

pipe.fit(X_train, y_train)

y_train_pred = pipe.predict(X_train)
rmse = np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))
r2_score = metrics.r2_score(y_train, y_train_pred)
print("Train rmse", rmse, "training r2_score", r2_score)

y_test_pred = pipe.predict(X_test)
rmse = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
r2_score = metrics.r2_score(y_test, y_test_pred)
print("Test rmse", rmse, "test r2_score", r2_score)

Train rmse 0.22021427332560542 training r2_score 0.6690890368374893
Test rmse 0.2489567011639626 test r2_score 0.6720482268615199


In [23]:
%%time
param_grid = {
    "poly__degree": [1, 2], 
    "est__alpha": 10 ** np.linspace(-2, 2, 10)
}

grid = model_selection.GridSearchCV(pipe, param_grid, cv = 5, n_jobs = 4,                           
        verbose = True, scoring = "neg_mean_squared_error")
grid.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
CPU times: user 4min 17s, sys: 33.2 s, total: 4min 50s
Wall time: 4min 49s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.8min finished


In [24]:
- grid.best_estimator_

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('est', Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))])

In [25]:
grid.best_score_

-0.024017117726390463

In [26]:
grid.best_params_

{'est__alpha': 0.01, 'poly__degree': 1}

In [27]:
np.linspace(-2, 2, 10)

array([-2.        , -1.55555556, -1.11111111, -0.66666667, -0.22222222,
        0.22222222,  0.66666667,  1.11111111,  1.55555556,  2.        ])